#### Package Import

In [1]:
import numpy as np
from numpy import math
from scipy.stats import norm
from scipy import stats
import matplotlib.pyplot as plt
import progressbar

#### Model Specification: OU Process
 1. $dX_{t} = \theta_{1}(\theta_{2} - X_{t})dt + \sigma dW_{t}$, $Y_{t}|X_{t} \sim \mathcal{N}(X_{t}, \theta_{3}^2)$
 2. $\mathbb{E}[X_{t}] = x_{0} e^{-\theta_1t} + \theta_{2} (1-e^{-\theta_{1}t})$, $Var[X_{t}] = \frac{\sigma^{2}}{2\theta_{1}}(1-e^{-2t\theta_1})$
 3. $Y_{1},Y_{2},...$ mutually independent, $Y_{t} \sim_{i.i.d.} \mathcal{N}(\mathbb{E}[X_{t}], \theta_{3}^2 + Var[X_{t}])$, for $t \in \mathbb{N}_{0}$

In [2]:
initial_val = 1
sigma = 0.5
theta = np.array([1,0,np.sqrt(0.2)])


def diff_coef(x, dt, dw):
    return sigma*np.math.sqrt(dt)*dw

def drift_coef(x, dt):
    return theta[0]*(theta[1]-x)*dt

# Log-scaled unnormalized likelihood function p(y|x)
def likelihood_logscale(y, x):
    d = (y-x)
    gn = -1/2*(d**2/(theta[2]**2))
    return gn

def likelihood_update(y,un,unormal_weight):
    gamma = math.sqrt(0.2)
    d = (y-un)
    gn1 = -1/2*(d**2/(theta[2]**2)) + unormal_weight
    return gn1

def sig_mean(t,theta):
    return initial_val*np.exp(-theta[0]*t) + theta[1]*(1-np.exp(-theta[0]*t))

## Used only when theta[0] != 0
def sig_var(t,theta):
    return (sigma**2 / (2*theta[0])) * (1-np.exp(-2*theta[0]*t))

def gen_data(T):
    Y = np.zeros(T+1)
    for t in range(T+1):
        std = np.sqrt(sig_var(t,theta) + theta[2]**2)
        Y[t] = sig_mean(t,theta) + std * np.random.randn(1)
    return Y

def Kalmanfilter(T,Y):
    
    m = np.zeros((T+1))
    mhat = np.zeros((T+1))
    c = np.zeros((T+1))
    a = theta[0]
    s = sigma
    # observational noise variance is gam^2*I
    gam = theta[2]
    # dynamics noise variance is sig^2*I
    sig = np.sqrt(s**2/2/a*(1-np.exp(-2*a)))
    # dynamics determined by A
    A = np.exp(-a)
    # initial mean&covariance
    m[0] = initial_val
    c[0] = 0
    H = 1
    # solution & assimilate!
    for t in range(T):
        mhat[t] = A*m[t] + theta[1]*(1-A)
        chat = A*c[t]*A + sig**2
        ########################
        d = Y[t+1] - H*mhat[t]
        # Kalmab Gain
        K = (chat*H) / (H*chat*H + gam**2)
        # Mean Update
        m[t+1] = mhat[t] + K*d
        # Covariance update
        c[t+1] = (1-K*H)*chat
    tv = m[T]
    return tv

def Kalmanfilter_path(T,Y):
    
    m = np.zeros((T+1))
    mhat = np.zeros((T+1))
    c = np.zeros((T+1))
    a = theta[0]
    s = sigma
    # observational noise variance is gam^2*I
    gam = theta[2]
    # dynamics noise variance is sig^2*I
    sig = np.sqrt(s**2/2/a*(1-np.exp(-2*a)))
    # dynamics determined by A
    A = np.exp(-a)
    # initial mean&covariance
    m[0] = initial_val
    c[0] = 0
    H = 1
    # solution & assimilate!
    for t in range(T):
        mhat[t] = A*m[t] + theta[1]*(1-A)
        chat = A*c[t]*A + sig**2
        ########################
        d = Y[t+1] - H*mhat[t]
        # Kalmab Gain
        K = (chat*H) / (H*chat*H + gam**2)
        # Mean Update
        m[t+1] = mhat[t] + K*d
        # Covariance update
        c[t+1] = (1-K*H)*chat
    return m

#### Main Function

In [3]:
# Resampling - input one-dimensional particle x
def resampling(weight, gn, x, N):
    ess = 1/((weight**2).sum())
    if ess <= (N/2):
        ## Sample with uniform dice
        dice = np.random.random_sample(N)
        ## np.cumsum obtains CDF out of PMF
        bins = np.cumsum(weight)
        ## np.digitize gets the indice of the bins where the dice belongs to 
        x_hat = x[np.digitize(dice,bins)]
        ## after resampling we reset the accumulating weight
        gn = np.zeros(N)
    if ess > (N/2):
        x_hat = x
    
    return x_hat, gn

# Coupled Wasserstein Resampling
def coupled_wasserstein(fine_weight, coarse_weight, gn, gc, fine_par, coarse_par, N):
    ess = 1/((fine_weight**2).sum())
    fine_hat = fine_par
    coarse_hat = coarse_par
    if ess <= (N/2):
        # Sort in ascending order of particles
        ind = np.argsort(fine_par[:])
        inc = np.argsort(coarse_par[:])
        fine_par = fine_par[ind]
        fine_weight = fine_weight[ind]
        coarse_par = coarse_par[inc]
        coarse_weight = coarse_weight[inc]
        # Sample with uniform dice
        dice = np.random.random_sample(N)
        # CDF
        bins = np.cumsum(fine_weight)
        bins1 = np.cumsum(coarse_weight)
        # get the indices of the bins where the dice belongs to
        fine_hat = fine_par[np.digitize(dice, bins)]
        coarse_hat = coarse_par[np.digitize(dice, bins1)]
        # reset accumulating weight after resampling
        gn = np.zeros(N)
        gc = np.zeros(N)
    if ess > (N/2):
        fine_hat = fine_par
        coarse_hat = coarse_par

    return fine_hat, gn, coarse_hat, gc

# Maixmally Coupled Resampling
def coupled_maximal(fine_weight, coarse_weight, gn, gc, fine_par, coarse_par, N):
    ess = 1/((fine_weight**2).sum())
    if ess <= (N/2):
        # Maximal coupled resampling
        fine_hat, coarse_hat = maximal_resample(fine_weight, coarse_weight, fine_par, coarse_par, N)
        # reset accumulating weight after resampling
        gn = np.zeros(N)
        gc = np.zeros(N)
    if ess > (N/2):
        fine_hat = fine_par
        coarse_hat = coarse_par
    return fine_hat, gn, coarse_hat, gc

def maximal_resample(weight1,weight2,x1,x2,N):
    # Initialize
    x1_hat = np.zeros(N)
    x2_hat = np.zeros(N)

    # Calculating many weights
    unormal_min_weight = np.minimum(weight1, weight2)
    min_weight_sum = np.sum(unormal_min_weight)
    min_weight = unormal_min_weight / min_weight_sum
    unormal_reduce_weight1 = weight1 - unormal_min_weight
    unormal_reduce_weight2 = weight2 - unormal_min_weight

    ## Sample with uniform dice
    dice = np.random.random_sample(N)
    ## [0] takes out the numpy array which is suitable afterwards
    coupled = np.where(dice <= min_weight_sum)[0]
    independ = np.where(dice > min_weight_sum)[0]
    ncoupled = np.sum(dice <= min_weight_sum)
    nindepend = np.sum(dice > min_weight_sum)
    
    if ncoupled>=0:
        dice1 = np.random.random_sample(ncoupled)
        bins = np.cumsum(min_weight)
        x1_hat[coupled] = x1[np.digitize(dice1,bins)]
        x2_hat[coupled] = x2[np.digitize(dice1,bins)]
   
    ## nindepend>0 implies min_weight_sum>0 imples np.sum(unormal_reduce_weight*) is positive, thus the division won't report error
    if nindepend>0:
        reduce_weight1 = unormal_reduce_weight1 / np.sum(unormal_reduce_weight1)
        reduce_weight2 = unormal_reduce_weight2 / np.sum(unormal_reduce_weight2)
        dice2 = np.random.random_sample(nindepend)
        bins1 = np.cumsum(reduce_weight1)
        bins2 = np.cumsum(reduce_weight2)
        x1_hat[independ] = x1[np.digitize(dice2,bins1)]
        x2_hat[independ] = x2[np.digitize(dice2,bins2)]
        
    return x1_hat, x2_hat


def Particle_filter(l,T,N,Y):
    hl = 2**(-l)
    un = np.zeros(N)+initial_val
    un_hat = un
    gn = np.zeros(N)
    for t in range(T):
        un_hat = un
        for dt in range(2**l):
            dw = np.random.randn(N)
            un = un + drift_coef(un, hl) + diff_coef(un, hl, dw)
        # Cumulating weight function    
        gn = likelihood_logscale(Y[t+1], un) + gn
        what = np.exp(gn-np.max(gn))
        wn = what/np.sum(what)
        
        # Wasserstein resampling
        un_hat, gn = resampling(wn, gn, un, N)
    
    return(np.sum(un*wn))


def Coupled_particle_filter_wasserstein(l,T,N,Y):
    hl = 2**(-l)
    ## Initial value
    un1 = np.zeros(N) + initial_val
    cn1 = np.zeros(N) + initial_val
    gn = np.ones(N)
    gc = np.ones(N)
    for t in range(T):
        un = un1
        cn = cn1
        for dt in range(2**(l-1)):
            dw = np.random.randn(2,N)
            for s in range(2):
                un = un + drift_coef(un, hl) + diff_coef(un, hl, dw[s,:])
            cn = cn + drift_coef(cn, hl*2) + diff_coef(cn, hl, (dw[0,:] + dw[1,:]))
        
        ## Accumulating Weight Function
        gn = likelihood_update(Y[t+1], un, gn)
        what = np.exp(gn-np.max(gn))
        wn = what/np.sum(what)
        gc = likelihood_update(Y[t+1], cn, gc)
        wchat = np.exp(gc-np.max(gc))
        wc = wchat/np.sum(wchat)
        ## Wassersteing Resampling 
        un1, gn, cn1, gc = coupled_wasserstein(wn,wc,gn,gc,un,cn,N)

    return(np.sum(un*wn-cn*wc))


def Coupled_particle_filter_maximal(l,T,N,Y):
    hl = 2**(-l)
    ## Initial value
    un1 = np.zeros(N) + initial_val
    cn1 = np.zeros(N) + initial_val
    gn = np.ones(N)
    gc = np.ones(N)
    for t in range(T):
        un = un1
        cn = cn1
        for dt in range(2**(l-1)):
            dw = np.random.randn(2,N)
            for s in range(2):
                un = un + drift_coef(un, hl) + diff_coef(un, hl, dw[s,:])
            cn = cn + drift_coef(cn, hl*2) + diff_coef(cn, hl, (dw[0,:] + dw[1,:]))
        
        ## Accumulating Weight Function
        gn = likelihood_update(Y[t+1], un, gn)
        what = np.exp(gn-np.max(gn))
        wn = what/np.sum(what)
        gc = likelihood_update(Y[t+1], cn, gc)
        wchat = np.exp(gc-np.max(gc))
        wc = wchat/np.sum(wchat)
        ## Wassersteing Resampling 
        un1, gn, cn1, gc = coupled_maximal(wn,wc,gn,gc,un,cn,N)

    return(np.sum(un*wn-cn*wc))

def coef(x, y): 
    # number of observations/points 
    n = np.size(x) 
  
    # mean of x and y vector 
    m_x, m_y = np.mean(x), np.mean(y) 
  
    # calculating cross-deviation and deviation about x 
    SS_xy = np.sum(y*x) - n*m_y*m_x 
    SS_xx = np.sum(x*x) - n*m_x*m_x 
  
    # calculating regression coefficients 
    b_1 = SS_xy / SS_xx 
    b_0 = m_y - b_1*m_x 
  
    return(b_0, b_1) 

#### Based on one Model and one Dataset, we need to fit:
1. $\mathbb{E}[(\eta_{t}^{l,N}(\varphi) - \eta_{t}^{l}(\varphi))^2] = C_{2} \frac{1}{N}$ 
2. $\mathbb{E}[\big((\eta_{t}^{l}-\eta_{t}^{l-1})^{N}(\varphi) - (\eta_{t}^{l}-\eta_{t}^{l-1})(\varphi)\big)^2] = C_{3} \frac{\Delta_{l}^{\beta}}{N}$

In [4]:
# Function to tune values of C_2
def fit_c2(data_path):
    rep_num = 100
    num_seq = np.zeros(6)
    var_pf = np.zeros(6)
    T = data_path.shape[0]-1
    for i in range(6):
        num_seq[i] = 100 * 2**i
        rep_val = np.zeros(rep_num)
        #pr = progressbar.ProgressBar(max_value=rep_num).start()
        for j in range(rep_num):
            rep_val[j] = Particle_filter(0,T,int(num_seq[i]),data_path)
            #pr.update(j+1)
        #pr.finish()
        print(i,'in 6 finished')
        var_pf[i] = np.var(rep_val)
        
    x = np.log10(num_seq)
    y = np.log10(var_pf)
    b=coef(x,y)
    print('slope is:',b[1])
    print('c2 value:',10**(b[0]))
    return 10**(b[0])

# Function to tune values of C_3, as well as values of beta
def fit_c3_beta(data_path):
    rep_num = 100
    N = 200
    l_seq = np.zeros(6)
    delt_seq = np.zeros(6)
    var_cpf = np.zeros(6)
    T = data_path.shape[0]-1
    for i in range(6):
        l_seq[i] = i+1
        delt_seq[i] = 2**(-(i+1))
        rep_val = np.zeros(rep_num)
        #pr = progressbar.ProgressBar(max_value=rep_num).start()
        for j in range(rep_num):
            rep_val[j] = Coupled_particle_filter_maximal(int(l_seq[i]),T,N,data_path)
            #pr.update(j+1)
        #pr.finish()
        print(i,'in 6 finished')        
        var_cpf[i] = np.var(rep_val)
        
    x = np.log10(delt_seq)
    y = np.log10(var_cpf)
    b=coef(x,y)
    beta = b[1]
    print('beta decimal value is:',b[1])
    print('c3 value:',10**(b[0])*N)
    return 10**(b[0])*N, round(b[1])

#### Parallel Particle Filter: Untuned

1. Choice of truncated distribution $\mathbb{P}_{P}(p) = 2^{-p}$ for $p \in \{0,1,...,P_{max}\}$, $\mathbb{P}_{L}(l) = 2^{-\beta l}$ for $l \in \{1,2,...,L_{max}\}$, $L_{max} = P_{max}$.
2. $N_{p} = 2^{p}N_{0}$, $N_{0} = C P_{max}^{2}2^{2P_{max}}$ $\Delta_{l}=2^{-l\beta}$
3. For MSE target of $\mathcal{O}(\epsilon^{2})$, we need cost of $\mathcal{O}(\epsilon^{-2} (\log(\epsilon))^{3})$ when $\beta=2$, $\mathcal{O}(\epsilon^{-2} (\log(\epsilon))^{4})$ when $\beta=1$ and $\mathcal{O}(\epsilon^{-2.5} (\log(\epsilon))^{3})$ when $\beta=\frac{1}{2}$.

#### Parallel Particle Filter:
1. if $l=0$, $N_{p}=C C_{2} P_{max}^{2}2^{2P_{max}}$ 
2. if $l>0$, $N_{p}=C C_{3} P_{max}^{2}2^{2P_{max}}$
3. The constant $C$ is tuned so that the MSE of the PPF estimator is of the same order (roughly twice) as its variance

In [16]:
def num_coupled_par(p, p_max, const):
    return int(2**(p+2*p_max) * (p_max**2) * const * c3)

def num_par(p, p_max, const):
    return int(2**(p+2*p_max) * (p_max**2) * const * c2)

def prob_l_func(max_val):
    prob = np.zeros(max_val)
    for l in range(max_val):
        prob[l] = 2**(-l*beta)
    prob = prob / np.sum(prob)
    return prob

def prob_p_func(max_val):
    prob = np.zeros(max_val)
    for p in range(max_val):
        prob[p] = 2**(-p)
    prob = prob / np.sum(prob)
    return prob

def Xi_zero(T,p_prob,p_max,const,Y):
    # sample the variable P
    p = int(np.random.choice(p_max, 1, p=p_prob)[0])
    #print('p_val is',p)
    # construct the estimator
    Xi_zero = (Particle_filter(0,T,num_par(p, p_max, const),Y) - Particle_filter(0,T,num_par(p-1, p_max, const),Y)) / p_prob[p]
    return Xi_zero

def Xi_nonzero(l,T,p_prob,p_max,const,Y):
    # sample the variable P
    p = int(np.random.choice(p_max, 1, p=p_prob)[0])
    #print('p_val is',p)
    # construct the estimator
    Xi = (Coupled_particle_filter_maximal(l,T,num_coupled_par(p,p_max,const),Y) - Coupled_particle_filter_maximal(l,T,num_coupled_par(p-1,p_max,const),Y)) / p_prob[p]
    return Xi

def Xi(T,l_prob,l_max,p_prob,p_max,const,Y):
    l = int(np.random.choice(l_max, 1, p=l_prob)[0])
    #print('value of l is',l)
    if l==0:
        Xi = Xi_zero(T,p_prob,p_max,const,Y)
    if l!=0:
        Xi = Xi_nonzero(l,T,p_prob,p_max,const,Y)
    est = Xi / l_prob[l]
    return est
    
def parallel_particle_filter(M,T,max_val,const,Y):
    l_max = max_val
    p_max = max_val
    l_prob = prob_l_func(l_max)
    p_prob = prob_p_func(p_max)
    est_summand = np.zeros(M)
    for m in range(M):
        est_summand[m] = Xi(T,l_prob,l_max,p_prob,p_max,const,Y)
    return (np.mean(est_summand))

def parallel_particle_filter_record(M,T,max_val,const,Y):
    l_max = max_val
    p_max = max_val
    l_prob = prob_l_func(l_max)
    p_prob = prob_p_func(p_max)
    est_summand = np.zeros(M)
    pr = progressbar.ProgressBar(max_value=M).start()
    for m in range(M):
        est_summand[m] = Xi(T,l_prob,l_max,p_prob,p_max,const,Y)
        pr.update(m+1)
    pr.finish()
    return est_summand

def Xi_zero_with_p(T,p_prob,p_max,const,Y):
    # sample the variable P
    p = int(np.random.choice(p_max, 1, p=p_prob)[0])
    #print('p_val is',p)
    # construct the estimator
    Xi_zero = (Particle_filter(0,T,num_par(p, p_max, const),Y) - Particle_filter(0,T,num_par(p-1, p_max, const),Y)) / p_prob[p]
    return Xi_zero, p

def Xi_nonzero_with_p(l,T,p_prob,p_max,const,Y):
    # sample the variable P
    p = int(np.random.choice(p_max, 1, p=p_prob)[0])
    #print('p_val is',p)
    # construct the estimator
    Xi = (Coupled_particle_filter_maximal(l,T,num_coupled_par(p,p_max,const),Y) - Coupled_particle_filter_maximal(l,T,num_coupled_par(p-1,p_max,const),Y)) / p_prob[p]
    return Xi, p

def Xi_with_pl(T,l_prob,l_max,p_prob,p_max,const,Y):
    l = int(np.random.choice(l_max, 1, p=l_prob)[0])
    #print('value of l is',l)
    if l==0:
        Xi, p_val = Xi_zero_with_p(T,p_prob,p_max,const,Y)
    if l!=0:
        Xi, p_val = Xi_nonzero_with_p(l,T,p_prob,p_max,const,Y)
    est = Xi / l_prob[l]
    return est, l, p_val

def cost_proxy_ppf(p_collect,l_collect, max_val, const):
    M = p_collect.shape[0]
    cost_proxy_val = 0
    for i in range(M):
        if l_collect[i] == 0:
            cost_proxy_val += num_par(p_collect[i], max_val, const)
        if l_collect[i] != 0:
            cost_proxy_val += num_coupled_par(p_collect[i], max_val, const) * 2**(l_collect[i])
    return cost_proxy_val

def parallel_particle_filter_record_with_cost(M,T,max_val,const,Y):
    l_max = max_val
    p_max = max_val
    l_prob = prob_l_func(l_max)
    p_prob = prob_p_func(p_max)
    est_summand = np.zeros(M)
    p_collect = np.zeros(M)
    l_collect = np.zeros(M)
    pr = progressbar.ProgressBar(max_value=M).start()
    for m in range(M):
        est_summand[m], p_collect[m], l_collect[m] = Xi_with_pl(T,l_prob,l_max,p_prob,p_max,const,Y)
        pr.update(m+1)
    pr.finish()
    
    cost_proxy_val = T * cost_proxy_ppf(p_collect,l_collect, max_val, const)
    
    return est_summand, cost_proxy_val

#### Multilevel Particle Filter: Untuned
1. When $\beta=2$, $N_{l} = 2^{2L-1.5l}$, to target a MSE of $\epsilon^{2}$, cost required is $\mathcal{O}(\epsilon^{-2})$
2. When $\beta=1$, $N_{l} = 2^{2L-l}L$, to target a MSE of $\epsilon^{2}$, cost required is $\mathcal{O}(\epsilon^{-2}(\log(\epsilon))^{2})$
3. When $\beta=\frac{1}{2}$, $N_{l} = 2^{2.25L - 0.75l}$, to target a MSE of $\epsilon^{2}$, cost required is $\mathcal{O}(\epsilon^{-2.5})$


#### Multilevel Particle Filter: 
1. When $\beta=2$, $N_{0} = C_{1}C_{2}2^{2L}$, $N_{l}=C_{1}C_{3}2^{2L-1.5l}$
2. When $\beta=1$, $N_{0}=C_{1}C_{2}2^{2L}L$, $N_{l}=C_{1}C_{3}2^{2L-l}L$
3. When $\beta=\frac{1}{2}$, $N_{0}=C_{1}C_{2}2^{2.25L}$, $N_{l}=C_{1}C_{3}2^{2.25L-0.75l}$
4. The constant $C_{1}$ is choosen such that the MSE of MLPF estimator $\eta_{t}^{L,N_{0:L}}$ is of same order (roughly twice) as its variance.

In [17]:
# For OU process, beta=2
def num_ml_coupled(l,lmax,const):
    return 2**(2*lmax-1.5*l) * const * c3

def num_ml_single(l,lmax,const):
    return 2**(2*lmax-1.5*l) * const * c2

def mlpf(T,max_val,const,Y):
    L = max_val
    level_est = np.zeros(L)
    level_est[0] = Particle_filter(0,T,int(num_ml_single(0,L,const)),Y)
    for l in range(1,L):
        level_est[l] = Coupled_particle_filter_maximal(l,T,int(num_ml_coupled(l,L,const)),Y)
    return np.sum(level_est)

def proxy_cost_mlpf(T,max_val,const):
    cost_proxy_val = 0
    cost_proxy_val += T*num_ml_single(0,max_val,const)
    for l in range(max_val):
        cost_proxy_val += T*num_ml_coupled(l,max_val,const) * 2**(l)
    return cost_proxy_val

#### Data set for specific OU model used in HPC implementaion:

In [5]:
T = 100
data_path = np.load('ou_model_data_path.npy')
c2, c3, beta = np.load('ou_fit_values.npy')

#### Experiment Example: Comparing MLPF with PPF on sigle processor
1. For a given $L_{max}$ values, for instance $L_{max}=2$, we first tune the constant $C$ for the PPF (parallel particle filter). We denote the PPF estimator as $\frac{1}{M}\sum_{i=1}^{M}\Xi^{i}$, on single processor, we assume $M=1$, we check the value of $Var(\Xi)=\mathcal{O}(C^{-1})$ for any initial guess on $C$ value, and then obtain the true $C$ by ensuring the variance of the PPF estimator is roughly equal to its squared bias. In this case, we should set $C=1000000$.
2. Computing Time of PPF estimator, this can be extremly costly to run, proxy cost represented by the number of Euler discretizations is used.
3. We include also cell to count the actual computing time for PPF, as for the HPC implementation, we will compare instead the actual computation time.

In [23]:
# Rep_num here is different from M, we record all the xi values and take variance
const = 1000
true_val = Kalmanfilter(T,data_path)
mse_seq = np.zeros(6)
var_seq = np.zeros(6)
square_bias_seq = np.zeros(6)
l_seq = np.arange(2,3)
rep_num = 1000
for i,lmax in enumerate(l_seq):
    est_val = parallel_particle_filter_record(rep_num, T, lmax, const, data_path)
    mse_seq[i] = np.mean((est_val-true_val)**2)
    var_seq[i] = np.var(est_val)
    square_bias_seq[i] = mse_seq[i] - var_seq[i] 
    print('At level',lmax,'Mse val:',mse_seq[i], 'Var val:', var_seq[i], 'Square Bias val:', square_bias_seq[i])

100% (1000 of 1000) |####################| Elapsed Time: 0:03:35 Time:  0:03:35


At level 2 Mse val: 0.0003864107054792948 Var val: 0.00038618142532472404 Square Bias val: 2.2928015457073956e-07


In [ ]:
# Actual Time for a single replication
lmax = 1
const = 1000000
%timeit parallel_particle_filter(1, T, lmax, const, data_path)

In [ ]:
# Proxy Time Cost for a single replication
lmax = 1
const = 1000000
cost_proxy_val = parallel_particle_filter_record_with_cost(1, T, lmax, const, data_path)[1]
print('Estimated Cost for PPF estimator:', cost_proxy_val)

4. In order to achieve similar MSE levels, we test and see that $L=6$ is required for MLPF estimator. In order to keep its variance to be roughly the same as its squared bias, we tune $C_{1}$ in a similar way. We can conlude that $C_{1}=600$ here.
5. Again the cost is evaluated through a proxy, the number of Euler discretizations involved to construct the MLPF estimator.
6. We include also cell to count the actual computing time for MLPF, as for the HPC implementation, we will compare instead the actual computation time.

In [30]:
const = 10
true_val = Kalmanfilter(T,data_path)
mse_seq = np.zeros(6)
var_seq = np.zeros(6)
square_bias_seq = np.zeros(6)
l_seq = np.arange(6,7)
rep_num = 1000
for i,lmax in enumerate(l_seq):    
    # repe of mlpf estimator
    est_val = np.zeros(rep_num)
    pr = progressbar.ProgressBar(max_value=rep_num).start()
    for j in range(rep_num):
        est_val[j] = mlpf(T,lmax,const,data_path)  
        pr.update(j+1)
    pr.finish()
    mse_seq[i] = np.mean((est_val-true_val)**2)
    var_seq[i] = np.var(est_val)
    square_bias_seq[i] = mse_seq[i] - var_seq[i] 
    print('At level',lmax,'Mse val:',mse_seq[i], 'Var val:', var_seq[i], 'Square Bias val:', square_bias_seq[i])

100% (1000 of 1000) |####################| Elapsed Time: 0:03:50 Time:  0:03:50


At level 6 Mse val: 6.483557920983263e-05 Var val: 6.382529355465471e-05 Square Bias val: 1.0102856551779163e-06


In [ ]:
# Actual Computing Time
const = 600
lmax = 6
%timeit mlpf(T,lmax,const,data_path)

In [ ]:
# Proxy Computation Time
const = 100
lmax = 6
proxy_cost_mlpf(T,lmax,const)